In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [49]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import confusion_matrix, classification_report

# !pip install category_encoders
from category_encoders import LeaveOneOutEncoder

<IPython.core.display.Javascript object>

# 🎄🌳🌴🌱🌲

☝️That's a pretty random forest

We're going to revisit the mammographic mass data set.  Details below.

Dataset from UCI can be found [here](http://archive.ics.uci.edu/ml/datasets/mammographic+mass).

1. BI-RADS assessment: 1 to 5 (ordinal)
2. Age: patient's age in years (integer)
3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
6. Severity: benign=0 or malignant=1 (binary)

## Data prep time!

In [3]:
data_url = "https://docs.google.com/spreadsheets/d/1d4TGnU2PYppNiRJIby7NQB2hfvWb8I8eyWWi2og_Zf4/export?format=csv"
columns = ["BI-RADS", "Age", "Shape", "Margin", "Density", "Severity"]

<IPython.core.display.Javascript object>

In [4]:
breast_cancer = pd.read_csv(data_url, names=columns)

<IPython.core.display.Javascript object>

This data encoded NaNs as `?`.  Convert the `?`s to NA and the columns to numeric.

In [5]:
breast_cancer = breast_cancer.replace("?", np.nan)
breast_cancer = breast_cancer.apply(pd.to_numeric, axis=1)

<IPython.core.display.Javascript object>

Drop NAs

In [6]:
breast_cancer.shape

(961, 6)

<IPython.core.display.Javascript object>

In [7]:
breast_cancer = breast_cancer.dropna()

<IPython.core.display.Javascript object>

In [8]:
X = breast_cancer.drop(columns=["Severity", "BI-RADS"])
y = breast_cancer["Severity"]



X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


<IPython.core.display.Javascript object>

Okie doke, from the description we had some 'nominal' (aka categorical columns).  We want to encode these.  The nominal columns are: `['Shape', 'Margin']`.

We're going to switch things up and use `category_encoders.LeaveOneOutEncoder` instead of `sklearn.preprocessing.OneHotEncoder`.  More on this encoder can be seen in the `leave_one_out_encoding.ipynb` notebook in this folder.

In [9]:
cat_cols = ["Shape", "Margin"]
num_cols = ["Age", "Density"]

<IPython.core.display.Javascript object>

In [10]:
encoder = LeaveOneOutEncoder(cols=cat_cols)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)

<IPython.core.display.Javascript object>

In [11]:
X_train

,Age,Shape,Margin,Density
646,68.0,0.783172,0.828283,3.0
802,31.0,0.161765,0.128906,3.0
439,46.0,0.783172,0.611765,3.0
826,60.0,0.783172,0.712195,3.0
722,66.0,0.783172,0.712195,3.0
...,...,...,...,...
123,66.0,0.176471,0.128906,3.0
170,21.0,0.161765,0.128906,3.0
354,83.0,0.783172,0.712195,3.0
544,67.0,0.783172,0.828283,3.0


<IPython.core.display.Javascript object>

Last bit of data prep is to separate out into our `X` and `y` components and `train_test_split()`.  We're predicting the `'Severity'` variable.

## Random Forest background
### Concept 1: Bootstrapping ☠️

Fancier name than method.  Bootstrapping is repeatedly sampling with replacement.

In [12]:
X_train.sample(n=3)

,Age,Shape,Margin,Density
873,36.0,0.783172,0.828283,3.0
602,63.0,0.783172,0.712195,3.0
750,64.0,0.176471,0.128906,3.0


<IPython.core.display.Javascript object>

Sample 3 rows from `X_train`.

In [13]:
X_sample = X_train.sample(n=3)

<IPython.core.display.Javascript object>

Select the same 3 rows from `y_train`

In [14]:
y_sample = y_train.loc[X_sample.index]

<IPython.core.display.Javascript object>

Let's write a function to do this for us.

In [15]:
def xy_sample(X, y, size, random_state=None):
    # Do stuff
    X_sample = X.sample(n=size, random_state=random_state)
    y_sample = y.loc[X_sample.index]
    return X_sample, y_sample

<IPython.core.display.Javascript object>

So all we want to do is repeat that a few times.

In [16]:
n_samples = 5
sample_size = 3

bootstrap_samples = []
# Fill in the for loop for us to iterate and make samples
# The number of samples we want to make is stored in n_samples
for i in range(n_samples):
    # Perform the sampling like we just did
    # Use the sample_size variable
    X_sample, y_sample = xy_sample(X=X_train, y=y_train, size=sample_size)

    # Store in a dictionary to have nice X y labels
    train_sample = {"X": X_sample, "y": y_sample}

    # Store all our samples together in a list
    bootstrap_samples.append(train_sample)


bootstrap_samples

[{'X':       Age     Shape    Margin  Density
  199  55.0  0.176471  0.128906      3.0
  866  76.0  0.783172  0.712195      3.0
  861  64.0  0.783172  0.712195      3.0,
  'y': 199    0.0
  866    1.0
  861    1.0
  Name: Severity, dtype: float64},
 {'X':       Age     Shape    Margin  Density
  607  55.0  0.176471  0.128906      3.0
  670  79.0  0.176471  0.712195      3.0
  223  79.0  0.783172  0.712195      3.0,
  'y': 607    0.0
  670    1.0
  223    1.0
  Name: Severity, dtype: float64},
 {'X':       Age     Shape    Margin  Density
  818  42.0  0.530303  0.128906      2.0
  783  69.0  0.161765  0.128906      3.0
  593  53.0  0.783172  0.828283      3.0,
  'y': 818    0.0
  783    1.0
  593    0.0
  Name: Severity, dtype: float64},
 {'X':       Age     Shape    Margin  Density
  61   60.0  0.783172  0.611765      3.0
  2    58.0  0.783172  0.828283      3.0
  738  53.0  0.783172  0.712195      3.0,
  'y': 61     1.0
  2      1.0
  738    0.0
  Name: Severity, dtype: float64},
 {'X

<IPython.core.display.Javascript object>

Boom 💥we're bonified bootstrappers.

### Concept 2: Bagging 💰

Kind of some overlap with concept 1....

<font color='red'>B</font><font color='blue'>AGGING</font> = <font color='red'>B</font>ootstrap <font color='blue'>AGG</font>regat<font color='blue'>ING</font>

* Step 1: Build a bunch of models on bootstrap samples
* Step 2: Aggregate the predictions of each model
* Step 3: dQw4w9WgXcQ
* Step 4: Profit

In [17]:
# Create a sample of size 10 like we've been doing
X_sample, y_sample = xy_sample(X_train, y_train, size=10, random_state=42)

# Fit a decision tree to this sample
tree_1 = DecisionTreeClassifier()
tree_1.fit(X_sample, y_sample)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

<IPython.core.display.Javascript object>

Second verse, same as the first.

In [18]:
# Create a sample of size 10 like we've been doing
X_sample, y_sample = xy_sample(X_train, y_train, size=10, random_state=1337)

# Fit a decision tree to this sample
tree_2 = DecisionTreeClassifier()
tree_2.fit(X_sample, y_sample)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

<IPython.core.display.Javascript object>

Again!

In [19]:
# Create a sample of size 10 like we've been doing
X_sample, y_sample = xy_sample(X_train, y_train, size=10, random_state=8675309)

# Fit a decision tree to this sample
tree_3 = DecisionTreeClassifier()
tree_3.fit(X_sample, y_sample)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

<IPython.core.display.Javascript object>

In [20]:
pred_1 = tree_1.predict(X_test)
pred_2 = tree_2.predict(X_test)
pred_3 = tree_3.predict(X_test)
pred_df = pd.DataFrame({"pred_1": pred_1, "pred_2": pred_2, "pred_3": pred_3})
pred_df

,pred_1,pred_2,pred_3
0,0.0,1.0,0.0
1,0.0,0.0,0.0
2,1.0,1.0,0.0
3,0.0,0.0,0.0
4,0.0,1.0,0.0
...,...,...,...
161,0.0,0.0,1.0
162,0.0,1.0,1.0
163,0.0,0.0,0.0
164,0.0,0.0,1.0


<IPython.core.display.Javascript object>

Who do we believe??  Let's be fair and just rulers, we'll take all our trees' votes into consideration like a true democracy.

In [21]:
pred_df.mean(axis=1)

0      0.333333
1      0.000000
2      0.666667
3      0.000000
4      0.333333
         ...   
161    0.333333
162    0.666667
163    0.000000
164    0.333333
165    0.333333
Length: 166, dtype: float64

<IPython.core.display.Javascript object>

In [22]:
pred_df["avg_vote"] = pred_df.mean(axis=1)
pred_df

,pred_1,pred_2,pred_3,avg_vote
0,0.0,1.0,0.0,0.333333
1,0.0,0.0,0.0,0.000000
2,1.0,1.0,0.0,0.666667
3,0.0,0.0,0.0,0.000000
4,0.0,1.0,0.0,0.333333
...,...,...,...,...
161,0.0,0.0,1.0,0.333333
162,0.0,1.0,1.0,0.666667
163,0.0,0.0,0.0,0.000000
164,0.0,0.0,1.0,0.333333


<IPython.core.display.Javascript object>

Convert the `'avg_vote'` column to a binary label.  Use 0.5 as a cutoff

In [23]:
pred_df["final_prediction"] = pred_df["avg_vote"] > 0.5
pred_df["final_prediction"] = pred_df["final_prediction"].astype(int)
pred_df

,pred_1,pred_2,pred_3,avg_vote,final_prediction
0,0.0,1.0,0.0,0.333333,0
1,0.0,0.0,0.0,0.000000,0
2,1.0,1.0,0.0,0.666667,1
3,0.0,0.0,0.0,0.000000,0
4,0.0,1.0,0.0,0.333333,0
...,...,...,...,...,...
161,0.0,0.0,1.0,0.333333,0
162,0.0,1.0,1.0,0.666667,1
163,0.0,0.0,0.0,0.000000,0
164,0.0,0.0,1.0,0.333333,0


<IPython.core.display.Javascript object>

What Percentage of the predictions are correct?

In [24]:
confusion_matrix(y_test, pred_df["final_prediction"])

array([[68, 21],
       [14, 63]], dtype=int64)

<IPython.core.display.Javascript object>

We just fit 3 pretty naive models.  I say naive because they each only saw 10 records, but there's strength in numbers! This is the idea behind bagging, each model sees a different side of the data so they have different 'experiences' and 'perspectives' on whats right and wrong.  By considering all of the 'opinions' equally we avoid overfitting and we're able to get higher accuracy (in general) than using a single model.

Here comes the downside...

When we did just 1 decision tree, we were able to plot a nice diagram of how it made its decisions.  In our example we just made 3 trees, we could plot each one, but trying to view all these decisions would be a lot.  So we just lost the nice intrepretability that came with a single tree.  In practice, we'll typically have more than 3 trees and this becomes harder and harder to explain (we'll see a way to deal with this).

### Concept 3: Random subspace 🌒

Our `X` component is sometimes referred to as our 'feature space'.  A 'subspace' is a subset of a 'space'.  So this fancy term just means that we'll be taking a sample of our columns.  We do this without replacement.

In [25]:
# X_train.sample?

<IPython.core.display.Javascript object>

In [27]:
X_train.sample(2, axis="columns")

,Margin,Age
646,0.828283,68.0
802,0.128906,31.0
439,0.611765,46.0
826,0.712195,60.0
722,0.712195,66.0
...,...,...
123,0.128906,66.0
170,0.128906,21.0
354,0.712195,83.0
544,0.828283,67.0


<IPython.core.display.Javascript object>

Well that wasn't too bad, but how does it fit into a random forest?  A random forest will only look at a few of the columns for each decision (i.e. a random subspace).  By doing this, we further protect against overfitting.  It's assuming that we want to learn patterns from every one of our features, if we happened to have a really powerful feature, we might end up only learning from it.  But with a random subspace, that powerful feature won't always be there as a crutch and so we're forced to learn from our other columns too.

So we just defined all the concepts of a random forest. Let's use one.

## Random Forests in action

Fit a random forest classifier to the data.

In [38]:
model = RandomForestClassifier(max_depth=5, n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

<IPython.core.display.Javascript object>

Print out the accuracy of the predictor on the training and test data.

In [39]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"train_score: {train_score}")
print(f"test_score: {test_score}")

train_score: 0.8448795180722891
test_score: 0.8313253012048193


<IPython.core.display.Javascript object>

Let's see more than just accuracy, how can we see a view of our true-positives, false-positives, etc.?

In [40]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.78      0.83        89
         1.0       0.78      0.90      0.83        77

    accuracy                           0.83       166
   macro avg       0.84      0.84      0.83       166
weighted avg       0.84      0.83      0.83       166



<IPython.core.display.Javascript object>

Based on this output, do we have higher precision or recall?  What `sklearn` function could we use to prove this?

### Importance for intepretability

The 'importances' are stored in the `feature_importances_` attribute of our model.  What does the trailing underscore mean?

In [41]:
model.feature_importances_

array([0.23993557, 0.37325299, 0.36511451, 0.02169692])

<IPython.core.display.Javascript object>

Store the importances in a dataframe with a column for each features name.

In [42]:
feat_imp = pd.DataFrame(
    {"feat": X_train.columns, "importance": model.feature_importances_}
)

<IPython.core.display.Javascript object>

Order the dataframe from most to least important.

In [44]:
feat_imp.sort_values("importance", ascending=False)

,feat,importance
1,Shape,0.373253
2,Margin,0.365115
0,Age,0.239936
3,Density,0.021697


<IPython.core.display.Javascript object>

So shape is the most important feature in determining if these mammographic masses are benign or malignant.  What does that mean?  Remember that each feature is only chosen if it's the best split available, and that the way this is chosen is based on the 'information gain'.  We have a lot of trees, and we aggregate these measures of information gain across all the trees to get importance.  So the more important a feature, the more useful it was in separating our 2 classes across all of our forest.

### Importance for feature selection

Our forest's feature importances are letting us know what is best to identify the classes.  Why not use these to indicate which features we should use in a model.

You might do this in the case that you'd like to use linear regression but you want to subset down to useful features.  This is sort of like a more manual LASSO, but linearity isn't considered in the selection.

We still need to consider multicollinearity, highly correlated features can lead to unreliable importance numbers.  (i.e. maybe the model finds temperature is really important, but it used a celsius column half the time and a farenheit column the other times, this would lower the importance of those 2 columns)

* Use `sklearn`'s `SelectFromModel` to select the best 3 features for predicting the target.

In [47]:
selector = SelectFromModel(RandomForestClassifier())

selector.fit(X_train, y_train)
selector.transform(X_train)

array([[68.        ,  0.78317152,  0.82828283],
       [31.        ,  0.16176471,  0.12890625],
       [46.        ,  0.78317152,  0.61176471],
       ...,
       [83.        ,  0.78317152,  0.71219512],
       [67.        ,  0.78317152,  0.82828283],
       [68.        ,  0.17647059,  0.12890625]])

<IPython.core.display.Javascript object>

In [48]:
col_filter = selector.get_support()
X.columns[col_filter]

Index(['Age', 'Shape', 'Margin'], dtype='object')

<IPython.core.display.Javascript object>

### Tuning Random Forest

We have the same hyperparameters as we did for decision trees.  In addition, we have a parameter for how many trees should be in our forest.

* Use `sklearn`'s `GridSearchCV` to choose the best combination of hyperparameters and fit a model 
* Evaluate the model's performance.

In [79]:
%%time
grid = {
    'n_estimators': [105, 115,125],
    'max_depth':[ 11,13,15],
    'min_samples_leaf':[35, 45, 55],
    'criterion':['gini', 'entropy']
}
model = GridSearchCV(RandomForestClassifier(), param_grid = grid, verbose=1, n_jobs=-1, cv=10)
model.fit(X_train, y_train)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 364 tasks      | elapsed:   11.6s


Wall time: 17.7 s


[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   17.5s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rand

<IPython.core.display.Javascript object>

In [80]:
model.best_params_

{'criterion': 'entropy',
 'max_depth': 11,
 'min_samples_leaf': 55,
 'n_estimators': 125}

<IPython.core.display.Javascript object>

In [81]:
model.score(X_train, y_train)

0.802710843373494

<IPython.core.display.Javascript object>

In [82]:
model.score(X_test, y_test)

0.8253012048192772

<IPython.core.display.Javascript object>